<pre><b>Author:</b> Ashlynn Wimer
<b>Date:</b>3/4/2024</pre>

This script was where I experimented with chatgpt as a labeling opion. In it, I fine tune a GPT model, test the performance of the fine tuned and untuned GPT at labeling data, before concluding (incorrectly due to a lack of knowledge of daily request limits) that GPT was the way forward for my data classification.

In [3]:
import pandas as pd
import numpy as np
import jsonlines as jsonl
from sklearn.metrics import classification_report

from openai import OpenAI
import openai
import os

import HelperChan
import GPTHelper
import warnings

We start by acquiring and formatting our data to be used for fine tuning.

In [4]:
# Read in labeled posts
labeled_posts = pd.concat(
    [
        pd.read_csv('../data/first_pass_labels.csv', index_col='Unnamed: 0'),
        pd.read_csv('../data/lgbt_week_2_classified.csv', index_col='Unnamed: 0'),
        pd.read_csv('../data/lgbt_week_3_classified.csv', index_col='Unnamed: 0')
    ],
    ignore_index=True
).drop_duplicates('id')

# Read in all posts to allow for backref
all_posts = pd.concat(
    [
        pd.read_csv('../data/lgbt_week_1.csv'), pd.read_csv('../data/lgbt_week_2.csv'), pd.read_csv('../data/lgbt_week_3.csv')
    ], ignore_index=True
).drop_duplicates('id')

with warnings.catch_warnings(action="ignore"):
    # Backref!!
    labeled_posts['backref'] = labeled_posts['content'].apply(lambda x: HelperChan.content_with_back_reference(str(x), all_posts))

# Recommended lower bound for significant performance improvement according to OpenAI
train = labeled_posts.sample(n=50)
test = labeled_posts.drop(index=train.index)

messages = GPTHelper.create_example_messages(train, content_col='backref')

The following sections contain the code I used to create and test a fine-tuned model to compare against base GPT3.5. Any commented out code is code that I do not want to risk running twice on accident. 

In [3]:
# Write our messages to a jsonl
# with jsonl.open('../data/fine_tuning/output.jsonl', mode='w') as writer:
#     for message in messages:
#         writer.write(message)

In [5]:
openai.api_key = os.environ['OPENAI_API_KEY2']

client = OpenAI(
    api_key=openai.api_key.strip()
)

# client.files.create(
#     file=open('../data/fine_tuning/output.jsonl', 'rb'),
#     purpose='fine-tune'
# )

The following code chunk was used to generate the fine tuning job. However, I *really* don't want to risk re-running the job, so it's commented out.

In [ ]:
# client.fine_tuning.jobs.create(
#      training_file='file-gT3eiC36253G1jdgP8b2I4GM',
#      model='gpt-3.5-turbo-0125',
#      hyperparameters={
#          'n_epochs':3
#      }
#)

We now get both base GPT and tuned GPT to label our posts.

In [6]:
def create_prompt_messages(posts, content_col='content', system_role=None):
    '''
    Given a dataframe of 4chan posts with content in their content_col,
    creates prompt messages for the posts.

    Inputs:
      posts (DataFrame): posts
      content_col (str): name of the column containing content. Defaults to 'content'
      system_role (str): name of the system_role, uses a prompt for classifiyn g4chan posts if None. Defautls None.

    Returns: list of prompt messages for GPT. 
    '''

    messages = []
    system_role = 'You are a classifier which reads the raw content of posts from 4chan\'s /lgbt/ board, and says whether or not they discuss trans people or trans-related topics. If the post is about trans people, respond "Yes". Otherwise, respond "No".'    
    for post in posts[content_col]:

        user_prompt = \
        f'''
        Does the following message discuss trans people or trans-related topics? Yes or No.
        
        MESSAGE: {post}
        '''

        message = \
            [
                {'role': 'system', 'content':system_role},
                {'role': 'user', 'content':user_prompt}
            ]
        
        messages.append(message)
    
    return messages

def call_gpt_on_prompts(client, prompts, model='gpt-3.5-turbo-0125'):
    '''
    Call GPT on a list of prompts and return a list of responses.
    
    Inputs:
      client (OpenAI): the client to call
      prompts (list): list of prompts
      model (str): list of dictionary prompts.

    Returns: list of completion messages
    '''
    responses = []
    for message in prompts:
        response = client.chat.completions.create(
            model=model,
            messages=message
        )
        responses.append(response.choices[0].message.content)
    
    return responses

def call_gpt_on_prompts_with_saves(client, prompts, dataframe, save_loc, 
                                   save_col='gptlabels', model='gpt-3.5-turbo-0125', 
                                   n_splits=10):
    '''
    Calls GPT on prompts and gets responses, but actually saves its
    progress as it proceeds.

    Inputs: 
      client (OpenAI): OpenAI client.
      prompts (list): list of prompts for OpenAI.
      dataframe (DataFrame): DataFrame outputs should be saved to.
      save_loc (str): string location of where this function should save
        its outputs to.
      save_col (str): string name of the desired column to save to.
      model (str): name of model to use.
      save_rate (int): how many promts should be completed between saves.

    Returns: None. Saves resultant dataframe to save_loc
    '''

    assert len(dataframe) == len(prompts), 'Prompts and batches are mismatched.'

    df_batches = np.array_split(dataframe, n_splits)
    prompts_batches = np.split(np.array(prompts), n_splits)
    prompts_batches = [prompt_batch.tolist() for prompt_batch in prompts_batches]

    attached_dfs = []
    for prompts, batch in zip(prompts_batches, df_batches):
        
        # Get and attach prompts
        resps = call_gpt_on_prompts(client, prompts, model)
        batch[save_col] = resps

        # Save
        attached_dfs.append(batch.copy())
        pd.concat(attached_dfs, ignore_index=True).to_csv(save_loc)

In [ ]:
prompts = create_prompt_messages(test, content_col='backref')

#untuned_resps = call_gpt_on_prompts(client, prompts)

Save our work in case we're cursed

In [ ]:
# test['untuned'] = untuned_resps

#test.to_csv('../data/test_with_untuned.csv')

In [ ]:
#tuned_resps = call_gpt_on_prompts(client, prompts, model='ft:gpt-3.5-turbo-0125:personal::8zCxjHNl')

In [ ]:
#test['tuned'] = tuned_resps

#test.to_csv('../data/test_with_both.csv')

In [10]:
test = pd.read_csv('../data/test_with_both.csv', index_col='Unnamed: 0')
test['untuned'] = test['untuned'].apply(lambda x: 1 if x=='Yes' else 0)
test['tuned'] = test['tuned'].apply(lambda x: 1 if x=='Yes' else 0)

In [11]:
test.head()

,subject,id,author,date,time,content,clean_content,refs,urls,classification,backref,untuned,tuned
0,post suifuel,34577209,Anonymous,2/7/2024,19:35:44,">>34571853If they stop after the first one, th...","\r\nIf they stop after the first one, they loo...",['>>34571853'],[],1,<ref>'How do these plastic surgery addicts loo...,1,1
1,34604558,34605860,)*Kassandra of Ellaphae|PSO2,2/10/2024,2:34:13,>>34605630thought someone might like that :3i ...,\r\nthought someone might like that :3i dont n...,['>>34605630'],[],0,<ref>'>>34605584that’s pretty hot actually'</r...,0,0
2,34606802,34611390,Anonymous,02/10/24,15:09:03,>>34606802>Is being a transbian bad?yesthey're...,\r\n>Is being a transbian bad?yesthey're all d...,['>>34606802'],[],1,"<ref>""Is being a transbian bad? Whenever I tel...",1,1
3,/chasergen/ black hole edition,34593140,Anonymous,2/9/2024,2:23:20,why do deranged transbians not stick to their ...,why do deranged transbians not stick to their ...,[],[],1,why do deranged transbians not stick to their ...,1,1
4,/mtfg/ huh?,34557268,"A.G.P. pilot ""naz"" Nullifier(...)",2/6/2024,1:44:31,heathers really resonates with mehttps://youtu...,heathers really resonates with mehttps://youtu...,[],['/watch?v=LIHPNqYiPSMalmost'],0,heathers really resonates with mehttps://youtu...,0,0


In [12]:
tuned_accuracy = (sum(test['tuned'] == test['classification'])) / len(test)
untuned_accuracy = (sum(test['untuned'] == test['classification'])) / len(test)

tuned_accuracy, untuned_accuracy

(0.88, 0.8622222222222222)

In [13]:
print(classification_report(test['classification'], test['tuned'], labels=[0,1]))

              precision    recall  f1-score   support

           0       0.87      0.92      0.89       244
           1       0.90      0.83      0.86       206

    accuracy                           0.88       450
   macro avg       0.88      0.88      0.88       450
weighted avg       0.88      0.88      0.88       450



In [14]:
print(classification_report(test['classification'], test['untuned'], labels=[0,1]))

              precision    recall  f1-score   support

           0       0.84      0.92      0.88       244
           1       0.89      0.80      0.84       206

    accuracy                           0.86       450
   macro avg       0.87      0.86      0.86       450
weighted avg       0.86      0.86      0.86       450



That is.. notably better than a supervised model performs. 

Guess that means I'm using GPT!

In [15]:
import warnings
from tqdm.auto import tqdm
tqdm.pandas()

with warnings.catch_warnings(action="ignore"):
    all_posts['backref'] = all_posts['content'].progress_apply(
        lambda x: HelperChan.content_with_back_reference(str(x), labeled_posts))

to_label = all_posts.sample(n=4500).reset_index(drop=True)

prompts = create_prompt_messages(to_label, content_col='backref')

c:\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
  0%|          | 0/186635 [00:00<?, ?it/s]

100%|██████████| 186635/186635 [00:26<00:00, 7033.27it/s]


In [16]:
call_gpt_on_prompts_with_saves(client, prompts=prompts, dataframe=to_label, 
                               save_loc='../data/gpt_labeled.csv', model='gpt-3.5-turbo-0125', 
                               n_splits=10)

c:\Python311\Lib\site-packages\numpy\core\fromnumeric.py:59: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)


In [18]:
labeled = pd.read_csv('../data/gpt_labeled.csv', index_col='Unnamed: 0')

already_labeled = all_posts['id'].isin(labeled['id'])

subset = all_posts[~already_labeled]

len(all_posts), len(labeled), len(subset)

(186635, 4500, 182135)

In [21]:
to_label = subset.sample(n=4500).reset_index(drop=True)

prompts = create_prompt_messages(to_label, content_col='backref')

In [22]:
call_gpt_on_prompts_with_saves(client, prompts=prompts, dataframe=to_label, 
                               save_loc='../data/gpt_labeled_2.csv', model='gpt-3.5-turbo-0125', 
                               n_splits=10)

c:\Python311\Lib\site-packages\numpy\core\fromnumeric.py:59: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)
